In [11]:
import pinocchio as pin
import os
from pinocchio.visualize import MeshcatVisualizer
import meshcat.geometry as g
import meshcat.transformations as tf

# Define the path to your URDF file
URDF_PATH = "/home/carlos/cp_unite_ws/src/robot_descriptions/bpl_bravo_description/urdf/bravo_5_dynamics_no_ee_pinocchio.urdf"

# Define the root directory of your robot package.
# This should be the directory that contains the 'bpl_bravo_description' folder.
# For example, if your URDF is at '/home/medusa/bravo7_compliance/reach-bravo-7/bpl_bravo_description/urdf/...',
# then the package directory is '/home/medusa/bravo7_compliance/reach-bravo-7/'.
PACKAGE_DIR = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(URDF_PATH))))

# The 'bpl_bravo_description' is the package name.
PACKAGE_NAME = "bpl_bravo_description"

# Load the URDF model, specifying the package directory
try:
    model, collision_model, visual_model = pin.buildModelsFromUrdf(
        URDF_PATH, package_dirs=[PACKAGE_DIR], meshLoader=pin.MeshLoader()
    )

    # Initialize the Meshcat visualizer
    from pinocchio.visualize import MeshcatVisualizer
    viz = MeshcatVisualizer(model, collision_model, visual_model)
    viz.initViewer(open=True)
    viz.loadViewerModel()

    # Display the robot in a neutral configuration
    q0 = pin.neutral(model)
    viz.display(q0)
    
except Exception as e:
    print(f"An error occurred: {e}")

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


Opening in existing browser session.


In [12]:
import numpy as np
STORE = np.array([4.904, 2.857, 0.05, 0.2304])
HOME = np.array([4.904, 2.592, 0.549, 0.0])

In [13]:
g.MeshLambertMaterial(
    color=0x00ff00,   # green
    transparent=True,
    opacity=0.3
)


In [14]:
model_amir = pin.buildModelFromUrdf(URDF_PATH)
data_amir = model_amir.createData()
tool_link = "contact_point"
frame_id_ee = model_amir.getFrameId("contact_point") 

In [15]:
last_frame_id = model_amir.nframes - 1
last_frame = model_amir.frames[last_frame_id]
last_frame

Frame name: contact_point paired to (parent joint/ parent frame)(4/14)
with relative placement wrt parent joint:
  R =
1 0 0
0 1 0
0 0 1
  p =    0    0 0.24
containing inertia:
  m = 0
  c = 0 0 0
  I = 
0 0 0
0 0 0
0 0 0

In [16]:
last_frame_id


15

In [17]:
frame_id_ee = model_amir.getFrameId("contact_point") 
frame_id_ee

15

In [18]:
viz.display(HOME)

In [19]:
import ipywidgets as widgets
from ipywidgets import interact

## Best simulation for differential inverse kinematics -- Analysis of force and motion manipulability

In [10]:
import numpy as np
import pinocchio as pin
import meshcat.geometry as g
import ipywidgets as widgets
from ipywidgets import interact

# Draw with U (principal directions) and radii already computed (no extra sqrt inside)
def draw_ellipsoid_from_svd(viz, path, U, radii, placement, color, opacity=0.4, scale=0.05):
    radii = np.asarray(radii, dtype=float)
    radii = np.maximum(radii, 1e-12)  # clamp to avoid zeros
    T = placement.copy()
    T[:3, :3] = T[:3, :3] @ U
    material = g.MeshLambertMaterial(color=color, transparent=True, opacity=opacity)
    viz.viewer[path].set_object(g.Ellipsoid(scale * radii), material)
    viz.viewer[path].set_transform(T)

# Global config (keep your q_current logic)
q_current = pin.neutral(model_amir)
q_current[:4] = HOME

# Sliders (we keep references so we can reset them)
slider_layout = widgets.Layout(width='80%')  # 600px wide
dx_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.001, value=0.0, layout=slider_layout)
dy_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.001, value=0.0, layout=slider_layout)
dz_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.001, value=0.0, layout=slider_layout)
rx_slider = widgets.FloatSlider(min=-0.2, max=0.2, step=0.001, value=0.0, layout=slider_layout)
ry_slider = widgets.FloatSlider(min=-0.2, max=0.2, step=0.001, value=0.0, layout=slider_layout)
rz_slider = widgets.FloatSlider(min=-0.2, max=0.2, step=0.001, value=0.0, layout=slider_layout)

def update_pose(dx=0.0, dy=0.0, dz=0.0, rx=0.0, ry=0.0, rz=0.0, scale_factor=0.05):
    global q_current

    # Use the real EE frame id; do NOT overwrite it with a hard-coded number
    frame_id = model_amir.getFrameId(tool_link)

    # FK/Jacobian at current config
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)
    J = pin.computeFrameJacobian(model_amir, data_amir, q_current, frame_id, pin.LOCAL)
    Jlin = J[:3, :]

    # Small damped IK step (more stable than raw pinv)
    dx_local = np.array([dx, dy, dz, rx, ry, rz])
    mu = 1e-4
    JJt = J @ J.T
    dq = J.T @ np.linalg.solve(JJt + (mu**2)*np.eye(6), dx_local)

    # Update only valid DOFs
    q_current[:len(dq)] += dq[:len(q_current)]

    # Update kinematics again after the move
    viz.display(q_current)
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)

    # ⬇️ SVD-based manipulability (clean and aligned)
    U, s, Vt = np.linalg.svd(Jlin, full_matrices=False)
    eps = 1e-12
    s = np.maximum(s, eps)
    motion_radii = s
    force_radii  = 1.0 / s

    # Place ellipsoids at the actual EE pose
    placement = data_amir.oMf[frame_id].homogeneous

    # Colors: motion = BLUE, force = RED
    draw_ellipsoid_from_svd(viz, "world/motion_ellipsoid", U, motion_radii, placement,
                            color=0x2609DE, opacity=0.45, scale=0.2)
    draw_ellipsoid_from_svd(viz, "world/force_ellipsoid", U, force_radii, placement,
                            color=0xFF0000, opacity=0.35, scale=0.02)

    # Quick sanity print
    print("singular values σ (motion radii):", s)
    print("1/σ (force radii):", 1.0/s)

    # 🔄 Reset sliders back to zero after applying the move
    dx_slider.value = 0.0
    dy_slider.value = 0.0
    dz_slider.value = 0.0
    rx_slider.value = 0.0
    ry_slider.value = 0.0
    rz_slider.value = 0.0

# Build interactive UI with fixed sliders
ui = widgets.VBox([dx_slider, dy_slider, dz_slider, rx_slider, ry_slider, rz_slider])
out = widgets.interactive_output(update_pose, 
                                 {'dx': dx_slider, 'dy': dy_slider, 'dz': dz_slider,
                                  'rx': rx_slider, 'ry': ry_slider, 'rz': rz_slider})

display(ui, out)


Output()

## IN WORLD AXIS

In [9]:
import numpy as np
import pinocchio as pin
import meshcat.geometry as g
import ipywidgets as widgets
from ipywidgets import interact

# Draw with U (principal directions) and radii already computed (no extra sqrt inside)
def draw_ellipsoid_from_svd(viz, path, U, radii, placement, color, opacity=0.4, scale=0.05):
    radii = np.asarray(radii, dtype=float)
    radii = np.maximum(radii, 1e-12)  # clamp to avoid zeros
    T = placement.copy()
    T[:3, :3] = T[:3, :3] @ U
    material = g.MeshLambertMaterial(color=color, transparent=True, opacity=opacity)
    viz.viewer[path].set_object(g.Ellipsoid(scale * radii), material)
    viz.viewer[path].set_transform(T)

# Global config (keep your q_current logic)
q_current = pin.neutral(model_amir)
q_current[:4] = HOME

# Sliders (we keep references so we can reset them)
slider_layout = widgets.Layout(width='80%')  # 600px wide
dx_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.001, value=0.0, layout=slider_layout)
dy_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.001, value=0.0, layout=slider_layout)
dz_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.001, value=0.0, layout=slider_layout)
rx_slider = widgets.FloatSlider(min=-0.2, max=0.2, step=0.001, value=0.0, layout=slider_layout)
ry_slider = widgets.FloatSlider(min=-0.2, max=0.2, step=0.001, value=0.0, layout=slider_layout)
rz_slider = widgets.FloatSlider(min=-0.2, max=0.2, step=0.001, value=0.0, layout=slider_layout)

def update_pose(dx=0.0, dy=0.0, dz=0.0, rx=0.0, ry=0.0, rz=0.0, scale_factor=0.05):
    global q_current

    # Use the real EE frame id; do NOT overwrite it with a hard-coded number
    frame_id = model_amir.getFrameId(tool_link)

    # FK/Jacobian at current config
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)
    J = pin.computeFrameJacobian(model_amir, data_amir, q_current, frame_id, pin.WORLD)
    Jlin = J[:3, :]

    # Small damped IK step (more stable than raw pinv)
    dx_local = np.array([dx, dy, dz, rx, ry, rz])
    mu = 1e-4
    JJt = J @ J.T
    dq = J.T @ np.linalg.solve(JJt + (mu**2)*np.eye(6), dx_local)

    # Update only valid DOFs
    q_current[:len(dq)] += dq[:len(q_current)]

    # Update kinematics again after the move
    viz.display(q_current)
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)

    # ⬇️ SVD-based manipulability (clean and aligned)
    U, s, Vt = np.linalg.svd(Jlin, full_matrices=False)
    eps = 1e-12
    s = np.maximum(s, eps)
    motion_radii = s
    force_radii  = 1.0 / s

    # Place ellipsoids at the actual EE pose
    placement = data_amir.oMf[frame_id].homogeneous

    # Colors: motion = BLUE, force = RED
    draw_ellipsoid_from_svd(viz, "world/motion_ellipsoid", U, motion_radii, placement,
                            color=0x2609DE, opacity=0.45, scale=0.2)
    draw_ellipsoid_from_svd(viz, "world/force_ellipsoid", U, force_radii, placement,
                            color=0xFF0000, opacity=0.35, scale=0.02)

    # Quick sanity print
    print("singular values σ (motion radii):", s)
    print("1/σ (force radii):", 1.0/s)

    # 🔄 Reset sliders back to zero after applying the move
    dx_slider.value = 0.0
    dy_slider.value = 0.0
    dz_slider.value = 0.0
    rx_slider.value = 0.0
    ry_slider.value = 0.0
    rz_slider.value = 0.0

# Build interactive UI with fixed sliders
ui = widgets.VBox([dx_slider, dy_slider, dz_slider, rx_slider, ry_slider, rz_slider])
out = widgets.interactive_output(update_pose, 
                                 {'dx': dx_slider, 'dy': dy_slider, 'dz': dz_slider,
                                  'rx': rx_slider, 'ry': ry_slider, 'rz': rz_slider})

display(ui, out)

Output()

# Only Position Jacobian

In [20]:
import numpy as np
import pinocchio as pin
import meshcat.geometry as g
import ipywidgets as widgets
from ipywidgets import interact

# Draw with U (principal directions) and radii already computed (no extra sqrt inside)
def draw_ellipsoid_from_svd(viz, path, U, radii, placement, color, opacity=0.4, scale=0.05):
    radii = np.asarray(radii, dtype=float)
    radii = np.maximum(radii, 1e-12)  # clamp to avoid zeros
    T = placement.copy()
    T[:3, :3] = T[:3, :3] @ U
    material = g.MeshLambertMaterial(color=color, transparent=True, opacity=opacity)
    viz.viewer[path].set_object(g.Ellipsoid(scale * radii), material)
    viz.viewer[path].set_transform(T)

# Global config (keep your q_current logic)
q_current = pin.neutral(model_amir)
q_current[:4] = HOME

# Sliders (we keep references so we can reset them)
slider_layout = widgets.Layout(width='80%')  # 600px wide
dx_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.001, value=0.0, layout=slider_layout)
dy_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.001, value=0.0, layout=slider_layout)
dz_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.001, value=0.0, layout=slider_layout)
rx_slider = widgets.FloatSlider(min=-0.2, max=0.2, step=0.001, value=0.0, layout=slider_layout)
ry_slider = widgets.FloatSlider(min=-0.2, max=0.2, step=0.001, value=0.0, layout=slider_layout)
rz_slider = widgets.FloatSlider(min=-0.2, max=0.2, step=0.001, value=0.0, layout=slider_layout)

def update_pose(dx=0.0, dy=0.0, dz=0.0, rx=0.0, ry=0.0, rz=0.0, scale_factor=0.05):
    global q_current

    # Use the real EE frame id; do NOT overwrite it with a hard-coded number
    frame_id = model_amir.getFrameId(tool_link)

    # FK/Jacobian at current config
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)
    J = pin.computeFrameJacobian(model_amir, data_amir, q_current, frame_id, pin.WORLD)
    Jlin = J[:3, :]

    # Small damped IK step (more stable than raw pinv)
    dx_local = np.array([dx, dy, dz])
    mu = 1e-4
    JJt = Jlin @ Jlin.T
    dq = Jlin.T @ np.linalg.solve(JJt + (mu**2)*np.eye(3), dx_local)

    # Update only valid DOFs
    q_current[:len(dq)] += dq[:len(q_current)]

    # Update kinematics again after the move
    viz.display(q_current)
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)

    # ⬇️ SVD-based manipulability (clean and aligned)
    U, s, Vt = np.linalg.svd(Jlin, full_matrices=False)
    eps = 1e-12
    s = np.maximum(s, eps)
    motion_radii = s
    force_radii  = 1.0 / s

    # Place ellipsoids at the actual EE pose
    placement = data_amir.oMf[frame_id].homogeneous

    # Colors: motion = BLUE, force = RED
    draw_ellipsoid_from_svd(viz, "world/motion_ellipsoid", U, motion_radii, placement,
                            color=0x2609DE, opacity=0.45, scale=0.2)
    draw_ellipsoid_from_svd(viz, "world/force_ellipsoid", U, force_radii, placement,
                            color=0xFF0000, opacity=0.35, scale=0.02)

    # Quick sanity print
    print("singular values σ (motion radii):", s)
    print("1/σ (force radii):", 1.0/s)

    # 🔄 Reset sliders back to zero after applying the move
    dx_slider.value = 0.0
    dy_slider.value = 0.0
    dz_slider.value = 0.0
    rx_slider.value = 0.0
    ry_slider.value = 0.0
    rz_slider.value = 0.0

# Build interactive UI with fixed sliders
ui = widgets.VBox([dx_slider, dy_slider, dz_slider, rx_slider, ry_slider, rz_slider])
out = widgets.interactive_output(update_pose, 
                                 {'dx': dx_slider, 'dy': dy_slider, 'dz': dz_slider,
                                  'rx': rx_slider, 'ry': ry_slider, 'rz': rz_slider})

display(ui, out)

Output()

## Best simulation for selected joint configuration -- Analysis of force and motion manipulability

In [12]:
def draw_joint_configuration_and_ellipsoids(joint_configuration):

    # Use the real EE frame id; do NOT overwrite it with a hard-coded number
    frame_id = model_amir.getFrameId(tool_link)

    # FK/Jacobian at current config
    pin.forwardKinematics(model_amir, data_amir, joint_configuration)
    pin.updateFramePlacements(model_amir, data_amir)
    J = pin.computeFrameJacobian(model_amir, data_amir, joint_configuration, frame_id, pin.LOCAL)
    Jlin = J[:3, :]

    mu = 1e-4
    JJt = J @ J.T


    # Update kinematics again after the move
    viz.display(joint_configuration)
    pin.forwardKinematics(model_amir, data_amir, joint_configuration)
    pin.updateFramePlacements(model_amir, data_amir)

    # ⬇️ SVD-based manipulability (clean and aligned)
    U, s, Vt = np.linalg.svd(Jlin, full_matrices=False)
    eps = 1e-12
    s = np.maximum(s, eps)
    motion_radii = s
    force_radii  = 1.0 / s

    # Place ellipsoids at the actual EE pose
    placement = data_amir.oMf[frame_id].homogeneous

    # Colors: motion = BLUE, force = RED (keep them consistent)
    draw_ellipsoid_from_svd(viz, "world/motion_ellipsoid", U, motion_radii, placement,
                            color=0x2609DE, opacity=0.45, scale=0.2)
    draw_ellipsoid_from_svd(viz, "world/force_ellipsoid", U, force_radii, placement,
                            color=0xFF0000, opacity=0.35, scale=0.02)

    # Quick sanity print
    print("singular values σ (motion radii):", s)
    print("1/σ (force radii):", 1.0/s)

def update_joint_position(theta1=4.904, theta2=2.592, theta3=0.549, theta4=0.0,  scale_factor=0.05):
    # update joint configuration (example: 3 DoF robot, extend as needed)
    q = pin.neutral(model_amir)
    q[0] = theta1
    q[1] = theta2
    q[2] = theta3
    q[3] = theta4

    viz.display(q)
    draw_joint_configuration_and_ellipsoids(q)


# --- interactive sliders ---
interact(update_joint_position,
         theta1=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=4.904),
         theta2=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=2.592),
         theta3=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=0.549),
         theta4=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=4.904),
         scale_factor=widgets.FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05))

interactive(children=(FloatSlider(value=4.904, description='theta1', max=10.0, min=-5.0), FloatSlider(value=2.…

<function __main__.update_joint_position(theta1=4.904, theta2=2.592, theta3=0.549, theta4=0.0, scale_factor=0.05)>